In [3]:
from IslandTime import Workflow, retrieve_island_info, TimeSeriesCoastSat, update_data_map, update_results_map, plot_shoreline_transects
import matplotlib.pyplot as plt
import Rbeast as rb
from datetime import datetime
import os
import numpy as np
from tqdm import tqdm
import matplotlib

matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
ii['timeseries_analysis']['coastline_position_transect_45_waterline']

In [19]:
ts = ii['timeseries_preprocessing']['optimal time period']['dict_timeseries']['coastline_position_transect_45_waterline']['monthly']['coastline_position_transect_45_waterline']
len(ts)
import Rbeast as rb

o = rb.beast(ts.values, deltat = '1/12 year', period='1 year')
rb.plot(o)

TypeError: 'int' object is not subscriptable

In [14]:
from statsmodels.tsa.stattools import acf, adfuller, kpss

plt.plot(ts.values - o.trend.Y)

In [33]:
plot_shoreline_transects(island_info)

In [34]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from matplotlib.collections import PolyCollection, LineCollection

t = 9
ts = island_info['timeseries_preprocessing']['optimal time period']['dict_timeseries']['coastline_position_transect_{}_waterline'.format(t)]['monthly']['coastline_position_transect_{}_waterline'.format(t)]

o = rb.beast(ts.values, deltat = '1/12 year', period='1 year')
# rb.plot(o)

def plot_acf_colors(ax, markercolor="#d62828", linecolor="black", facecolor="silver", barcolor="#003049", linewidth=1):
    """utility function to get some control over colors with  plot_acf()"""
    
    for item in ax.collections:
        # change the color of the confidence interval 
        if type(item) == PolyCollection:
            item.set_facecolor(facecolor)
        # change the color of the vertical lines
        if type(item) == LineCollection:
            item.set_color(barcolor)
    # change the color of the markers
    [line.get_label() for line in ax.lines]
    for item in ax.lines:
        item.set_color(markercolor)
    # change the color of the horizontal lines
    ax.lines[0].set_color(linecolor)
    ax.lines[0].set_linewidth(linewidth)
    #ax.lines.remove(ax.lines[0])
    return ax

fig, axs = plt.subplots(2,2, figsize=(20,20))
axs = axs.ravel()
plot_acf(ts - o.trend.Y, lags=40, alpha=0.05, ax=axs[1], color='k', use_vlines=True, fft=True, vlines_kwargs={'colors': 'k', 'linestyles': 'dashed'}, title='', gapcolor='k')
ax = plot_acf_colors(axs[1])
# axs[1].axvline(x=12, color='k', linestyle='-', linewidth=0.5, zorder=0, alpha=0.5)
# axs[1].axvline(x=6, color='k', linestyle='-', linewidth=0.5, zorder=0, alpha=0.5)
# axs[1].axvline(x=18, color='k', linestyle='-', linewidth=0.5, zorder=0, alpha=0.5)
# ax.axvline(x=24, color='red', linestyle='--')
# ax.axvline(x=36, color='red', linestyle='--')
# ax.axvline(x=6, color='g', linestyle='--', label='6 months')
# ax.axvline(x=18, color='g', linestyle='--')
# ax.axvline(x=30, color='g', linestyle='--')
axs[1].set_xlabel('Lag (months)', fontsize=20)
axs[1].set_ylabel('Autocorrelation', fontsize=20)
# axs[1].text(12.2, 0.75, '12 months', fontsize=15, fontweight='bold')
# axs[1].text(6.2, 0.85, '6 months', fontsize=15, fontweight='bold')
# axs[1].text(18.2, 0.65, '18 months', fontsize=15, fontweight='bold')
axs[1].set_xlim(-1, 40.5)
axs[1].set_ylim(-1.05, 1.05)
# Increase tick size
# We change the fontsize of minor ticks label 
axs[1].tick_params(axis='both', which='major', labelsize=12)
axs[3].tick_params(axis='both', which='major', labelsize=12)
# axs[1].tick_params(axis='both', which='minor', labelsize=20)

# axs[1].plot(ts, color='k', linewidth=1.5, label='Preprocessed time series')
# axs[1].set_xlabel('Time', fontsize=20)
# axs[1].set_ylabel('Coastline position (m)', fontsize=20)
# axs[1].legend()
# plot_shoreline_transects(island_info, ax=axs[0], transect_plot=10)

axs[3].plot(ts - o.trend.Y, color='#456990', linewidth=1.5, alpha=0.7, label='Preprocessed time series')
axs[3].set_xlabel('Time', fontsize=20)
axs[3].set_ylabel('Coastline position (m)', fontsize=20)
# ax.set_title('')
# ax.legend(fontsize=15)

import numpy as np
import skimage.exposure as exposure
import pyproj

def _image_stretch(im):
    # Rescale image for visualisation
    if np.ndim(im) > 2:
        for i in range(np.shape(im)[2]):
            im_no_nan = im[:, :, i][~np.isnan(im[:, :, i])]
            im[:, :, i] = exposure.rescale_intensity(im[:, :, i], in_range=(im_no_nan.min(), im_no_nan.max()))
    
    else:
        im_no_nan = im[~np.isnan(im)]
        im = exposure.rescale_intensity(im, in_range=(im_no_nan.min(), im_no_nan.max()))
    
    return im

kk = 73
# georef = all_dict[2][list(all_dict[2].keys())[kk]]
# rgb = all_dict[1][list(all_dict[1].keys())[kk]]

# axs[0].imshow(_image_stretch(rgb), extent=[georef[0], georef[0] + georef[1] * rgb.shape[1], georef[3] + georef[5] * rgb.shape[0], georef[3]])
plot_shoreline_transects(island_info, transect_plot=t, ax=axs[0])
axs[0].axis('off')

from scipy.fft import fft
from scipy.signal import find_peaks, argrelextrema
from scipy import stats


time_series = ts
# Fourier transform of time series
fft_result = fft(time_series.values - o.trend.Y)

# Get frequencies
frequencies = np.fft.fftfreq(len(time_series.values), d=1)#1/12)

# Only take the positive half of the spectrum
positive_freq = frequencies[:len(frequencies)//2]
positive_magnitude = np.abs(fft_result[:len(fft_result)//2])

# Convert frequency to cycles per year
positive_freq_yearly = positive_freq * 12

axs[2].plot(positive_freq_yearly[1:], positive_magnitude[1:], color='#20a39e', linewidth=1.5, label='Fast Fourier Transform')
axs[2].set_xlabel('Frequency (cycles per year)', fontsize=20)
axs[2].set_ylabel('Amplitude', fontsize=20)
axs[2].axvline(x=1, color='k', linestyle='-', linewidth=0.5, zorder=0, alpha=0.5)
axs[2].axvline(x=2, color='k', linestyle='-', linewidth=0.5, zorder=0, alpha=0.5)
axs[2].axvline(x=3, color='k', linestyle='-', linewidth=0.5, zorder=0, alpha=0.5)
axs[2].tick_params(axis='both', which='major', labelsize=12)
# axs[2].set_ylim(0, 600)
axs[2].set_xlim(0.1, 5)

# fft_result without frequency 0
fft_result_n = fft_result[1:]

# Remove frequencies with amplitude below a threshold
#print(all(element is False for element in (stats.zscore(np.abs(fft_result_n)) > self.z_score_seasonality)))
score_list = stats.zscore(np.abs(fft_result_n)) > 2.
if all(not element for element in score_list):
    fft_result = fft_result
else:
    threshold = np.min(np.abs(fft_result_n[stats.zscore(np.abs(fft_result_n)) > 2.]))
    fft_result[np.abs(fft_result) < threshold] = 0

# Only take the positive half of the spectrum
positive_freq = frequencies[:len(frequencies)//2]
positive_magnitude = np.abs(fft_result[:len(fft_result)//2])

# Convert frequency to cycles per year
positive_freq_yearly = positive_freq * 12

axs[2].plot(positive_freq_yearly[1:], positive_magnitude[1:], color='#ffba49', linewidth=2, ls = '--', label='Filtered Fast Fourier Transform')
axs[2].legend(fontsize=15)

# Inverse Fourier transform (filetered data)
filtered_data_fourier = np.fft.ifft(fft_result).real

# Find peaks
peaks_fourier, _ = find_peaks(filtered_data_fourier, height=0)

# Find period
period_fourier = stats.mode(np.diff(peaks_fourier)).mode
print(period_fourier)

axs[3].plot(time_series.index, filtered_data_fourier, color='#540d6e', linewidth=1.5, label='Fourier-filtered time series')
ii=0
for peak in peaks_fourier:
        if ii==0:
            axs[3].axvline(time_series.index[peak], color='#EF767A', linestyle='--', linewidth=1.3)
        else:
            axs[3].axvline(time_series.index[peak], color='#EF767A', linestyle='--', linewidth=1.3)
        ii+=1

import datetime
# axs[3].annotate('', xy=(datetime.datetime(2017, 5, 1), 136), xytext=(datetime.datetime(2018, 4, 1), 136), arrowprops=dict(arrowstyle='<->', color='black'))
# plt.text(datetime.datetime(2017, 10, 15), 138, '12 months', horizontalalignment='center', verticalalignment='center', fontsize=20, fontweight='bold')

# axs[3].annotate('', xy=(datetime.datetime(2017, 7, 1), 115), xytext=(datetime.datetime(2018, 1, 15), 115), arrowprops=dict(arrowstyle='<->', color='black'))
# plt.text(datetime.datetime(2017, 10, 1), 117, '7 months', horizontalalignment='center', verticalalignment='center', fontsize=20, fontweight='bold')

# axs[3].annotate('', xy=(datetime.datetime(2017, 2, 1), 95), xytext=(datetime.datetime(2017, 7, 1), 95), arrowprops=dict(arrowstyle='<->', color='black'))
# plt.text(datetime.datetime(2017, 4, 1), 97, '5 months', horizontalalignment='center', verticalalignment='center', fontsize=20, fontweight='bold')

axs[3].legend(fontsize=15, loc='best')
# ax[1, 1].set_title('Fourier reconstruction'

# fig.savefig('figures//non_seasonal_signal_new.png', dpi=300)

# axs[0].text()


INFO: To supress printing the parameers in beast(),      set print.options = 0 
INFO: To supress printing the parameers in beast_irreg(),set print.options = 0 
INFO: To supress printing the parameers in beast123(),   set extra.printOptions = 0  
INFO: To supress warning messages in beast(),            set quiet = 1 
INFO: To supress warning messages in beast_irreg(),      set quiet = 1 
INFO: To supress warning messages in beast123(),         set extra.quiet = 1  

#--------------------------------------------------#
#       Brief summary of Input Data                #
#--------------------------------------------------#
Data Dimension: One signal of length 65
IsOrdered     : Yes, ordered in time
IsRegular     : Yes, evenly spaced at interval of  0.0833333 year = 1 months = 30.4167 days
HasSeasonCmpnt: True  | period = 1 year = 12 months = 365 days. The model 'Y=Trend+Season+Error' is fitted.
              : Num_of_DataPoints_per_Period = period/deltaTime = 1/0.0833333 = 12
HasOutlier

In [2]:
ff = 'c:\\Users\\myriampe\\OneDrive - Imperial College London\\IslandTimeGitHub\\IslandTime\\data\\coastsat_data\\Kalherehaa_Maldives\\L8\\meta\\2013-11-14-05-22-20_L8_Kalherehaa_Maldives.txt'

with open(ff, 'rb') as f:
    print(f.read())

b'filename\t2013-11-14-05-22-20_L8_Kalherehaa_Maldives_ms.tif\r\nepsg\t32643\r\nacc_georef\t7.7\r\nimage_quality\t9\r\nim_width\t135\r\nim_height\t98\r\n'


In [3]:
plot_shoreline_transects(island_info)

In [4]:
from IslandTime import retrieve_island_info, Segmentation, update_results_map, plot_shoreline_transects, Workflow, update_data_map, TimeSeriesCoastSat, PreTimeSeries, TimeSeriesERA5, TimeSeriesClimateIndices
 
for island in ['Kereddhoo', ]: # Dhigelaabadhoo # Gaddhoo # Gan (Gaafu Dhaalu)  ## 'Kondeyviligili' 
 
    n, w, s, e = 0.633257, 73.339587, 0.627212, 73.349113

    # polygon = shapely.geometry.box(w, s, e, n)
    polygon = [[w, s], [e, s], [e, n], [w, n], [w, s]]


    # island_info = retrieve_island_info(island, 'Maldives', verbose=True)

    # PreTimeSeries(island, 'Maldives', polygon=polygon).main()
    # TimeSeriesCoastSat(island, 'Maldives', reference_shoreline_transects_only=True, overwrite=True, retrieve_reference_shoreline_manually=True).main()
    # TimeSeriesERA5(island, country='Maldives').main()
    # TimeSeriesClimateIndices(island, country='Maldives').main()
    # PreTimeSeries(island, 'Maldives', polygon=polygon).main()
    # 
    ii = Workflow(island, 'Maldives', run_all=False, execute_analysis=True, execute_segmentation=False, execute_preprocess=False, update_maps=False, overwrite_analysis=True).main()


-------------------------------------------------------------------
Retrieving all information available for the island
Island: Kereddhoo, Maldives
-------------------------------------------------------------------

~ The following information is available: ~

general_info
               island
               country
               atoll
spatial_reference
               latitude
               longitude
               polygon
               reference_shoreline
               transects
               area_country
               reference_shoreline_buffer_L8
               reference_shoreline_buffer_L9
               reference_shoreline_buffer_S2
image_collection_dict
               description
               S2
               L7
               L8
               L9
timeseries_coastsat
               description
               description_timeseries
               source
               inputs
               settings
               timeseries
timeseries_CRW
               description
   

Transect:   0%|          | 0/83 [00:00<?, ?it/s]

Transect:   1%|          | 1/83 [00:00<01:07,  1.22it/s]

Transect:   2%|▏         | 2/83 [00:01<01:18,  1.03it/s]

Transect:   4%|▎         | 3/83 [00:03<01:26,  1.08s/it]

...

Transect:  99%|█████████▉| 82/83 [01:24<00:00,  1.15it/s]

Transect: 100%|██████████| 83/83 [01:25<00:00,  1.03s/it]



-------------------------------------------------------------------
Retrieving all information available for the island
Island: Hadahaahuttaa, Maldives
-------------------------------------------------------------------

~ The following information is available: ~

general_info
               island
               country
               atoll
spatial_reference
               latitude
               longitude
               polygon
               reference_shoreline
               transects
               area_country
               reference_shoreline_buffer_L8
               reference_shoreline_buffer_L9
               reference_shoreline_buffer_S2
image_collection_dict
               description
               S2
               L7
               L8
               L9
timeseries_coastsat
               description
               description_timeseries
               source
               inputs
               settings
               timeseries
timeseries_CRW
               description

Transect:   0%|          | 0/159 [00:00<?, ?it/s]

: 

In [ ]:
from IslandTime import retrieve_island_info, Segmentation, update_results_map, plot_shoreline_transects, Workflow, update_data_map, TimeSeriesCoastSat, PreTimeSeries, TimeSeriesERA5, TimeSeriesClimateIndices

island = "Gaddhoo" # Dhigelaabadhoo # Gaddhoo # Gan (Gaafu Dhaalu)

n, w, s, e = 0.259568, 73.321894, 0.246963, 73.341152

# polygon = shapely.geometry.box(w, s, e, n)
polygon = [[w, s], [e, s], [e, n], [w, n], [w, s]]


# island_info = retrieve_island_info(island, 'Maldives', verbose=True)

# PreTimeSeries(island, 'Maldives', polygon=polygon).main()
# TimeSeriesCoastSat(island, 'Maldives', reference_shoreline_transects_only=True).main()
# TimeSeriesERA5(island, country='Maldives').main()
# TimeSeriesClimateIndices(island, country='Maldives').main()
# PreTimeSeries(island, 'Maldives', polygon=polygon).main()

ii = Workflow(island, 'Maldives', run_all=False, execute_analysis=True, execute_segmentation=False, execute_preprocess=False, update_maps=False, overwrite_analysis=True).main()


-------------------------------------------------------------------
Retrieving all information available for the island
Island: Gan (Gaafu Dhaalu), Maldives
-------------------------------------------------------------------

~ The following information is available: ~

general_info
               island
               country
               part of
               located in the administrative territorial entity
               located in or next to body of water
               elevation above sea level
               atoll
spatial_reference
               latitude
               longitude
               polygon
               polygon_OSM
               reference_shoreline
               transects
               area_country
               reference_shoreline_buffer_L8
               reference_shoreline_buffer_L9
               reference_shoreline_buffer_S2
image_collection_dict
               description
               S2
               L7
               L8
               L9
timeserie

Transect:   0%|          | 0/258 [00:00<?, ?it/s]

: 

In [18]:
update_results_map('Maldives')

Updating results maps...


100%|██████████| 406/406 [04:10<00:00,  1.62it/s]


In [13]:
# For legend
dict_colours_labels = {'sw_monsoon': '#58508d',
                        'ne_monsoon': '#bc5090', 
                        'transition_sw_ne': '#ffa600',
                        'transition_ne_sw': '#ff6361',
                        'undetermined': '#003f5c'}

list(dict_colours_labels.values())[0]

'#58508d'

In [ ]:
island = 'Hulhudhoo (South Maalhosmadulu)'

island_info = retrieve_island_info(island, 'Maldives')

In [ ]:
island_info['timeseries_analysis'].keys()

In [ ]:
for file in tqdm(os.listdir(os.path.join(os.getcwd(), 'data', 'info_islands'))):
    island = file.split('_')[1] 
    country = file.split('_')[2].split('.')[0]

    island_info = retrieve_island_info(island, country, verbose=False)

    if 'timeseries_analysis' in island_info.keys():
        if 'conditions_seasonality' in island_info['timeseries_analysis']['coastline_position_transect_0_waterline']['seasonality'].keys():
            continue
        
        else:
            island_info = Workflow(island, country, run_all=False, execute_analysis=True, execute_preprocess=False, execute_segmentation=False, update_maps=False, small_island=False).main()
            break

    else:
        continue

    # except:
    #     print('Error with island:', island)
    #     continue

In [ ]:
island = "Kan'duvilin'gili" 
country = 'Maldives'

# Retrieve island info
# island_info = retrieve_island_info(island, country, verbose=False)
# ii = TimeSeriesCoastSat(island, country, distance_between_transects=5, reference_shoreline_transects_only=True, overwrite=True, retrieve_reference_shoreline_manually=True).main()

# plot_shoreline_transects(island_info)

ii = Workflow(island, country, run_all=False, execute_analysis=True, execute_preprocess=False, execute_segmentation=False, update_maps=False, small_island=False, overwrite_analysis=True).main()

In [36]:
import pandas as pd

# Path to data
path_to_data = os.path.join(os.getcwd(), 'data', 'info_islands')

# Read file for islands to ignore
df_islands_ignore = pd.read_excel('islands_to_ignore.xlsx')
list_islands_ignore = df_islands_ignore['Island'].values

print('Updating results maps...')

for file in tqdm(os.listdir(path_to_data)):
    island = file.split('_')[1] 
    country = file.split('_')[2].split('.')[0]

    if island in list_islands_ignore:
        continue

    try:
        island_info = retrieve_island_info(island, country, verbose=False)
    
    except:
        print(f'Could not retrieve information for {island} in {country}')
        continue

Updating results maps...


  0%|          | 1/406 [00:00<00:55,  7.34it/s]

Could not retrieve information for Aakiraahuttaa in Maldives


  6%|▌         | 25/406 [00:12<02:27,  2.59it/s]

Could not retrieve information for Burunee in Maldives


 11%|█         | 44/406 [00:22<04:31,  1.33it/s]

In [31]:
list_atolls = ['Huvadhu', 'Addu']
fig, ax = plt.subplots()

for atoll in list_atolls:
    area = ox.geocode_to_gdf('{} Atoll, Maldives'.format(atoll))
    area.plot(ax=ax, facecolor='none', edgecolor='black')
plt.plot()

[]

In [ ]:
import osmnx as ox
import matplotlib.pyplot as plt
list_atolls = ['Huvadhu', 'Addu', 'North Male', 'Haa Alifu']
fig, ax = plt.subplots()

for atoll in list_atolls:
    print(atoll)
    area = ox.geocode_to_gdf('{}, Maldives'.format(atoll))
    area.plot(ax=ax, facecolor='none', edgecolor='black')
plt.show()

In [62]:
ox.geocode_to_gdf('Male, Maldives')

,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,class,type,place_rank,importance,addresstype,name,display_name
0,"MULTIPOLYGON (((73.45459 4.18088, 73.45471 4.1...",4.241171,4.168304,73.551915,73.454595,221321871,relation,6599336,4.177988,73.510739,boundary,administrative,8,0.542149,city,Malé,"Malé, Maldives"


In [50]:
import osmnx as ox
import matplotlib.pyplot as plt

maldives = ox.geocode_to_gdf('Maldives')
area = ox.geocode_to_gdf('Huvadhu Atoll, Maldives')

fig, ax = plt.subplots()
maldives.exterior.plot(ax=ax)
area.plot(ax=ax)
plt.show()

In [ ]:
island = 'Kodegalaa'
country = 'Maldives'

from IslandTime import Segmentation

Segmentation()

In [38]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# Create a figure and axis with a specific projection
fig, ax = plt.subplots(subplot_kw={'projection': ccrs.PlateCarree()})

# Set extent to focus on Maldives
ax.set_extent([72, 74, 2, 4])

# Add coastlines and gridlines
ax.coastlines()
ax.gridlines()

# Add a marker for Maldives
ax.plot(73.2207, 3.2028, 'ro', markersize=7, transform=ccrs.PlateCarree())
ax.text(73.2207 + 0.1, 3.2028 - 0.1, 'Maldives', transform=ccrs.PlateCarree())

# Show the plot
plt.show()

In [2]:
import osmnx as ox

area = ox.geocode_to_gdf('Huvadhu Atoll, Maldives')

In [ ]:
area.plot()

In [151]:
update_results_map('Maldives')

Updating results maps...


 86%|████████▌ | 343/401 [01:22<00:12,  4.80it/s]

Error with island: Qaruh Island


100%|██████████| 401/401 [01:31<00:00,  4.37it/s]


In [ ]:
import pandas as pd
with open('islands_to_ignore.txt') as f:
    lines = f.read().split('\n')
    arr = []
    for line in lines:
        arr.append(line.split(','))
    df = pd.DataFrame(arr, columns=['Island', 'Country', 'Atoll', 'Reason']) 
df.to_excel('islands_to_ignore.xlsx', index=False)

In [ ]:
df = pd.read_excel('islands_to_ignore.xlsx')

In [ ]:
'Fulu' in df.Island.values

In [148]:
from selenium import webdriver
import os

# Set up Selenium
options = webdriver.ChromeOptions()
options.add_argument('headless')  # To run Chrome in headless mode
driver = webdriver.Chrome(options=options)

# Open the HTML file in the browser
html_file_path = os.path.join(os.getcwd(), 'maps', 'results_islands_seasonality_minima.html')
driver.get(html_file_path)

# Wait for the map to load
import time
time.sleep(5)  # Adjust the time as needed

# Set the desired width and height for the window
desired_width = 1920
desired_height = 1080
driver.set_window_size(desired_width, desired_height)

# Take a screenshot
screenshot_path = 'map.png'
driver.save_screenshot(screenshot_path)

# Close the browser
driver.quit()

In [149]:
from selenium import webdriver

# Set up Selenium
options = webdriver.ChromeOptions()
options.add_argument('headless')  # To run Chrome in headless mode
driver = webdriver.Chrome(options=options)

# Open the HTML file in the browser
driver.get(os.path.join(os.getcwd(), 'maps', 'results_islands_seasonality_minima.html'))  # Replace with the path to your HTML file
# Wait for the map to load
import time
time.sleep(15)  # Adjust the time as needed

# Take a screenshot
driver.save_screenshot('maps//results_islands_seasonality_minima.png')

# Close the browser
driver.quit()

In [407]:
island = 'Dhakandhoo'
country = 'Maldives'

# island_info = retrieve_island_info(island, country)
# plot_shoreline_transects(island_info)

# island_info = TimeSeriesCoastSat(island, country, reference_shoreline_transects_only=True, overwrite=True, distance_between_transects=5, retrieve_reference_shoreline_manually=True).main()

island_info = Workflow(island, country, run_all=False, execute_segmentation=True, execute_preprocess=False, execute_analysis=False, update_maps=False, small_island=True).main()


-------------------------------------------------------------------
Retrieving all information available for the island
Island: Dhakandhoo, Maldives
-------------------------------------------------------------------

~ The following information is available: ~

general_info
               island
               country
               part of
               located in the administrative territorial entity
               located in or next to body of water
               elevation above sea level
               atoll
spatial_reference
               latitude
               longitude
               polygon
               polygon_OSM
               reference_shoreline
               transects
               transects_direction
               area_country
               reference_shoreline_buffer_L8
               reference_shoreline_buffer_L9
               reference_shoreline_buffer_S2
image_collection_dict
               description
               S2
               L5
               L7


MovieWriter Pillow unavailable; using Pillow instead.


In [ ]:
for file in tqdm(os.listdir(os.path.join(os.getcwd(), 'data', 'info_islands'))):
    island = file.split('_')[1] 
    country = file.split('_')[2].split('.')[0]

    if island == 'Dhigelaabadhoo' or island == 'Olhurataa':
        continue
    
    try:
        island_info = retrieve_island_info(island, country, verbose=False)
    except:
        print('Error with island:', island)
        continue
    if 'timeseries_analysis' in island_info.keys():
        ts_analysis_results = island_info['timeseries_analysis']
        # print(island)
        if 'amplitude_seasonal_STL_absrange' in ts_analysis_results[list(ts_analysis_results.keys())[0]]['seasonality'].keys():
            continue
        else:
            try:
                island_info = Workflow(island, country, run_all=False, execute_segmentation=True, execute_preprocess=True, execute_analysis=True, update_maps=False).main()
            except:
                print('Error with island:', island)
                continue
    else:
        continue

In [ ]:
# for file in tqdm(os.listdir(os.path.join(os.getcwd(), 'data', 'info_islands'))):
#     island = file.split('_')[1] 
#     country = file.split('_')[2].split('.')[0]
#     try:
#         island_info = Workflow(island, country, run_all=False, execute_segmentation=True, execute_preprocess=True, execute_analysis=True, update_maps=False).main()
#     except:
#         print('Error with island:', island)
#         continue

island = 'Keredhdhoo'
country = 'Maldives'

island_info = retrieve_island_info(island, country)

# TimeSeriesCoastSat(island, country, overwrite=True, reference_shoreline_transects_only=True, distance_between_transects=10).main()

#island_info = Workflow(island, country, run_all=False, execute_segmentation=False, execute_preprocess=False, execute_analysis=True, update_maps=False).main()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

# Get the Viridis colormap
viridis_cmap = plt.get_cmap('viridis')

# Number of colors to extract
num_colors = 7

# Sample the colormap at evenly spaced intervals
colors = [viridis_cmap(i / (num_colors - 1)) for i in range(num_colors)]

# Convert RGB colors to hexadecimal format (F code)
hex_colors = [f"#{int(color[0] * 255):02X}{int(color[1] * 255):02X}{int(color[2] * 255):02X}" for color in colors]

# Print the hexadecimal representation of the colors
for i, hex_color in enumerate(hex_colors):
    print(f"Color {i + 1}: {hex_color}")

In [ ]:
island_info['timeseries_analysis']['coastline_position_transect_100_waterline']['seasonality']['amplitude_seasonal_STL_absrange']

In [ ]:
ts_analysis_results = island_info['timeseries_analysis']
c_seasonality_amplitude = [ts_analysis_results[val]['seasonality']['amplitude_seasonal_STL_absrange'] for val in ts_analysis_results.keys()]

In [ ]:
c_seasonality_amplitude

In [ ]:
ts = island_info['timeseries_preprocessing']['optimal time period']['dict_timeseries']['coastline_position_transect_0_waterline']['monthly']['coastline_position_transect_0_waterline']
plt.plot(ts.index, ts.values)

o = rb.beast(ts.values, start=datetime.strftime(ts.index[0], '%Y-%m-%d'), season='monthly', deltat='1/12 year', quiet=True, print_progress=False)

In [ ]:
ts.index[0]

In [ ]:
plt.plot(ts.index, o.trend.Y)

# Maps climate data

In [1]:
import cdsapi
import urllib.request
import xarray as xr
import osmnx as ox

In [3]:
geometry_ox_country = ox.geocode_to_gdf('Maldives')
area_country = [geometry_ox_country.bbox_north.values[0], \
                geometry_ox_country.bbox_west.values[0], \
                geometry_ox_country.bbox_south.values[0], \
                geometry_ox_country.bbox_east.values[0]]

cds = cdsapi.Client(url="https://cds.climate.copernicus.eu/api/v2", key="200721:d13b27b3-32f8-4315-a9c0-e65dc3eb6fdd")

# Query cdsapi request
fl = cds.retrieve(
    'reanalysis-era5-single-levels-monthly-means',
    {
        'format': 'netcdf',
        'product_type': 'monthly_averaged_reanalysis',
        'variable': ['10m_u_component_of_wind', '10m_v_component_of_wind',
                        '2m_dewpoint_temperature', '2m_temperature', 
                        'soil_temperature_level_1', 'total_precipitation',
                        'evaporation', 'sea_surface_temperature', 'mean_sea_level_pressure',
        ],
        'time': '00:00',
        'year': [
            '2010', '2011', '2012', 
            '2013', '2014', '2015',
            '2016', '2017', '2018',
            '2019', '2020', '2021',
            '2022'
        ],
        'month': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ],
        'area': area_country,
    })

# Query cdsapi request
fl_waves = cds.retrieve(
    'reanalysis-era5-single-levels-monthly-means',
    {
        'format': 'netcdf',
        'product_type': 'monthly_averaged_reanalysis',
        'variable': ['mean_direction_of_total_swell', 'mean_direction_of_wind_waves', 
                        'mean_period_of_total_swell', 'mean_period_of_wind_waves', 
                        'mean_wave_direction', 'mean_wave_period',
                        'significant_height_of_combined_wind_waves_and_swell',
                        'significant_height_of_total_swell', 'significant_height_of_wind_waves',
        ],
        'time': '00:00',
        'year': [
            '2010', '2011', '2012', 
            '2013', '2014', '2015',
            '2016', '2017', '2018',
            '2019', '2020', '2021',
            '2022'
        ],
        'month': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ],
        'area': area_country,
    })

2024-07-15 15:40:41,221 INFO Welcome to the CDS
2024-07-15 15:40:41,223 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels-monthly-means
2024-07-15 15:40:41,635 INFO Request is queued
2024-07-15 15:40:42,675 INFO Request is running
2024-07-15 15:47:00,669 INFO Request is completed
2024-07-15 15:47:00,705 INFO Welcome to the CDS
2024-07-15 15:47:00,706 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels-monthly-means
2024-07-15 15:47:00,886 INFO Request is queued
2024-07-15 15:47:01,926 INFO Request is running
2024-07-15 15:48:54,647 INFO Request is completed


In [4]:
# Open file as xarray dataset
with urllib.request.urlopen(fl.location) as f:
    data_ERA5 = xr.open_dataset(f.read())

with urllib.request.urlopen(fl_waves.location) as f:
    data_ERA5_waves = xr.open_dataset(f.read())

In [6]:
huvadhu_atoll = ox.geocode_to_gdf('Huvadhu Atoll, Maldives')
huvadhu_atoll_box = [huvadhu_atoll.bbox_north.values[0], \
                huvadhu_atoll.bbox_west.values[0], \
                huvadhu_atoll.bbox_south.values[0], \
                huvadhu_atoll.bbox_east.values[0]]

In [9]:
data_ERA5_waves['mwd'].sel(time='2015-01-01').plot()

In [5]:
clip_data_waves = data_ERA5_waves.sel(latitude=slice(1.50, 0.), longitude=slice(72.6, 74.))

wave_height = clip_data_waves['swh']
wave_period = clip_data_waves['mwp']

# SI units
rho =  1025 # kg/m^3
g = 9.81 # m/s^2

# Wave energy flux in watts per meter of wave front
wave_energy = (rho * (g ** 2) * (wave_height ** 2) * wave_period) / (64 * np.pi)

#clip_data_waves['wave_energy'] = wave_energy

In [6]:
clip_data_waves['wave_energy'] = wave_energy

In [10]:
fig, ax = plt.subplots()
clip_data_waves['wave_energy'].sel(time='2015-01-01').plot(ax=ax)
huvadhu_atoll.plot(ax=ax, facecolor='none', edgecolor='black')

<Axes: title={'center': 'time = 2015-01-01'}, xlabel='longitude [degrees_east]', ylabel='latitude [degrees_north]'>

In [7]:
list_datasets = []
file_paths = os.path.join(os.getcwd(), 'data', 'copernicus_data')
list_files = os.listdir(file_paths)

for file in list_files:

    ds = xr.open_dataset(os.path.join('data', 'copernicus_data', file))
    list_datasets.append(ds)

    ds.close()

In [8]:
merged_dataset = xr.concat(list_datasets, dim='time')

In [20]:
merged_dataset

<xarray.Dataset> Size: 4GB
Dimensions:           (time: 211, nv: 2, latitude: 720, longitude: 1440)
Coordinates:
  * time              (time) datetime64[ns] 2kB 2005-01-15 ... 2022-07-15
  * latitude          (latitude) float32 3kB -89.88 -89.62 ... 89.62 89.88
  * longitude         (longitude) float32 6kB 0.125 0.375 0.625 ... 359.6 359.9
  * nv                (nv) int32 8B 0 1
Data variables:
    crs               (time) int32 844B -2147483647 -2147483647 ... -2147483647
    climatology_bnds  (time, nv) datetime64[ns] 3kB 2005-01-01 ... 2022-07-31
    lat_bnds          (time, latitude, nv) float32 1MB -89.75 -90.0 ... 89.75
    lon_bnds          (time, longitude, nv) float32 2MB 0.0 0.25 ... 359.8 360.0
    sla               (time, latitude, longitude) float64 2GB nan nan ... nan
    eke               (time, latitude, longitude) float64 2GB nan nan ... nan
Attributes: (12/43)
    Conventions:                     CF-1.6
    Metadata_Conventions:            Unidata Dataset Discovery v1.0
    cdm_data_type:                   Grid
    comment:                         Monthly Mean of Sea Level Anomalies refe...
    contact:                         http://climate.copernicus.eu/c3s-user-se...
    creator_email:                   http://climate.copernicus.eu/c3s-user-se...
    ...                              ...
    summary:                         Delayed Time Level-4 monthly means of Se...
    time_coverage_duration:          P1M
    time_coverage_end:               2005-01-31T00:00:00Z
    time_coverage_resolution:        P1M
    time_coverage_start:             2005-01-01T00:00:00Z
    title:                           DT merged two-satellite Global Ocean L4 ...

In [20]:
fig, ax = plt.subplots()
huvadhu_atoll.plot(ax=ax, facecolor='none', edgecolor='black')
merged_dataset['sla'].sel(latitude=slice(0., 1.0), longitude=slice(72.6, 74.), time='2022-07-15').plot(ax=ax)


In [21]:
# sea level
# List of file paths to the .nc files you want to merge
file_paths = os.path.join(os.getcwd(), 'data', 'copernicus_data')

# Open each file as an xarray dataset
datasets = [xr.open_dataset(file) for file in file_paths]

# Concatenate along a specific dimension (e.g., time)
merged_dataset = xr.concat(datasets, dim='time')

# Save the merged dataset to a new .nc file
# merged_dataset.to_netcdf('merged_file.nc')

# Close the datasets
for ds in datasets:
    ds.close()

ValueError: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'h5netcdf', 'scipy', 'pydap', 'zarr']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html
https://docs.xarray.dev/en/stable/user-guide/io.html

In [9]:
wave_energy = clip_data_waves['wave_energy']
wave_direction = clip_data_waves['mwd']
sea_level = merged_dataset['sla']

In [24]:
fig, ax = plt.subplots()
wave_energy.sel(time='2015-01-01').plot(ax=ax)
huvadhu_atoll.plot(ax=ax)

<Axes: title={'center': 'time = 2015-01-01'}, xlabel='longitude [degrees_east]', ylabel='latitude [degrees_north]'>

In [ ]:
# NE Monsoon: December to February
# NE-SW Transition: March to April
# SW Monsoon: May to September
# SW-NE Transition: October to November

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Sample data for wave direction and energy
wave_energy = clip_data_waves['wave_energy'].values.flatten()
wave_direction = clip_data_waves['mwd'].values.flatten() # Assuming wave_direction is already in degrees

# Create subplots
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})

# Plot wave energy (bar plot)
bars = ax.bar(np.radians(wave_direction), wave_energy, width=0.1, color='skyblue', alpha=0.7, label='Wave Energy')

# Set the theta zero location and direction
ax.set_theta_zero_location('N')
ax.set_theta_direction(-1)

# Adjust legend
lines, labels = ax.get_legend_handles_labels()
ax.legend(lines, labels, loc='upper right')

plt.title('Wave Direction and Energy')
plt.show()

In [13]:
sea_level

<xarray.DataArray 'sla' (time: 211, latitude: 720, longitude: 1440)> Size: 2GB
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])
Coordinates:
  * time       (time) datetime64[ns] 2kB 2005-01-15 2005-02-15 ... 2022-07-15
  * latitude   (latitude) float32 3kB -89.88 -89.62 -89.38 ... 89.38 89.62 89.88
  * longitude  (longitude) float32 6kB 0.125 0.375 0.625 ... 359.4 359.6 359.9
Attributes:
    cell_methods:   time: mean within years
    grid_mapping:   crs
    long_name:      Averaged Sea Level Anomalies 2005/01
    standard_name:  sea_surface_height_above_sea_level
    units:          m

In [27]:
fig, ax = plt.subplots()
clip_data_waves['wave_energy'].isel(latitude=slice(0, 2), longitude=slice(0,2), time=5).plot(ax=ax)
huvadhu_atoll.plot(ax=ax)

<Axes: title={'center': 'time = 2010-06-01'}, xlabel='longitude [degrees_east]', ylabel='latitude [degrees_north]'>

In [28]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

idxx = 1
idxy = 1

# Sample data for wave direction and energy
wave_energy = clip_data_waves['wave_energy'].isel(latitude=idxy, longitude=idxx).values.flatten()
wave_direction = clip_data_waves['mwd'].isel(latitude=idxy, longitude=idxx).values.flatten() # Assuming wave_direction is already in degrees

# Create subplots
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})

# Choose a colormap
cmap = plt.get_cmap('plasma')  # You can choose any other colormap

# Normalize wave energy values to range [0, 1]
norm = mcolors.Normalize(vmin=min(wave_energy), vmax=max(wave_energy))

# Plot wave energy using a colormap
bars = ax.bar(np.radians(wave_direction), wave_energy, width=0.1, color=cmap(norm(wave_energy)), alpha=0.7, label='Wave Energy')

# Add color bar for wave energy
cbar = plt.colorbar(plt.cm.ScalarMappable(cmap=cmap, norm=norm), ax=ax, label='Wave Energy')

# Set the theta zero location and direction
ax.set_theta_zero_location('N')
ax.set_theta_direction(-1)

# Adjust legend
#lines, labels = ax.get_legend_handles_labels()
#ax.legend(lines, labels, loc='upper right')

plt.title('Wave Direction and Energy')
plt.show()

In [29]:
clip_data_waves['wave_energy'].isel(latitude=idxy, longitude=idxx).sel(time=clip_data_waves['wave_energy'].time.dt.month.isin([4, 11]))

<xarray.DataArray 'wave_energy' (time: 26)> Size: 104B
array([3489.3508, 4391.886 , 3124.0706, 3514.175 , 3490.5405, 3346.449 ,
       2979.1653, 3107.802 , 2906.5706, 3955.991 , 2953.9219, 3641.015 ,
       2787.0837, 5241.4614, 3531.147 , 3676.1902, 3636.993 , 3402.801 ,
       3240.2876, 3404.3054, 3055.867 , 3511.7139, 3252.9915, 4944.2266,
       2848.115 , 3955.1824], dtype=float32)
Coordinates:
    longitude  float32 4B 73.36
    latitude   float32 4B 0.593
  * time       (time) datetime64[ns] 208B 2010-04-01 2010-11-01 ... 2022-11-01

In [30]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define the values for idxx and idxy
idxx_values = [0, 1, 2]
idxy_values = [0, 1, 2]

# Create a figure and axes for the joint plot
fig, axs = plt.subplots(3, 3, subplot_kw={'projection': 'polar'})

# Choose a common colormap for all subplots
cmap = plt.get_cmap('plasma')

# Normalize wave energy values to range [0, 1]
global_min_wave_energy = float('inf')
global_max_wave_energy = float('-inf')

# Iterate over idxx and idxy values to create the subplots
for idxx in idxx_values:
    for idxy in idxy_values:
        # Sample data for wave direction and energy
        wave_energy_ne_monsoon = clip_data_waves['wave_energy'].isel(latitude=idxy, longitude=idxx).sel(time=clip_data_waves['wave_energy'].time.dt.month.isin([1, 2, 3, 12])).values.flatten()
        wave_direction_ne_monsoon = clip_data_waves['mwd'].isel(latitude=idxy, longitude=idxx).sel(time=clip_data_waves['wave_energy'].time.dt.month.isin([1, 2, 3, 12])).values.flatten() # Assuming wave_direction is already in degrees

        wave_energy_sw_monsoon = clip_data_waves['wave_energy'].isel(latitude=idxy, longitude=idxx).sel(time=clip_data_waves['wave_energy'].time.dt.month.isin([4, 5, 6, 7, 8, 9, 10, 11])).values.flatten()
        wave_direction_sw_monsoon = clip_data_waves['mwd'].isel(latitude=idxy, longitude=idxx).sel(time=clip_data_waves['wave_energy'].time.dt.month.isin([4, 5, 6, 7, 8, 9, 10, 11])).values.flatten() # Assuming wave_direction is already in degrees

        # Calculate global min and max for wave_energy
        local_min_wave_energy = np.min(wave_energy)
        local_max_wave_energy = np.max(wave_energy)
        if local_min_wave_energy < global_min_wave_energy:
            global_min_wave_energy = local_min_wave_energy
        if local_max_wave_energy > global_max_wave_energy:
            global_max_wave_energy = local_max_wave_energy

        # Plot wave energy using a colormap
        ax = axs[idxy, idxx]
        bars = ax.bar(np.radians(wave_direction), wave_energy, width=0.1, color=cmap((wave_energy - global_min_wave_energy) / (global_max_wave_energy - global_min_wave_energy)), alpha=0.7, label='Wave Energy')

        # Set the theta zero location and direction
        ax.set_theta_zero_location('N')
        ax.set_theta_direction(-1)

# Add a common color bar for wave energy
norm = mcolors.Normalize(vmin=global_min_wave_energy, vmax=global_max_wave_energy)
cbar = fig.colorbar(plt.cm.ScalarMappable(cmap=cmap, norm=norm), ax=axs, orientation='vertical', label='Wave Energy')

# Adjust layout
#plt.tight_layout()

plt.show()

TypeError: strptime() argument 1 must be str, not numpy.datetime64

In [47]:
import pandas as pd
d = wave_energy.sel(latitude=0.5, longitude=73.1, method='nearest').time[0].values
# transform d from numpy.datetime64 to datetime
d = pd.to_datetime(d)

ow = rb.beast(wave_energy.sel(latitude=0.5, longitude=73.1, method='nearest').values, start = [d.year, d.month, d.day], season='monthly', deltat='1/12 year', quiet=True, print_progress=False)
# rb.plot(o)



In [60]:
import pandas as pd
oc = rb.beast(ts.values, start=datetime.strftime(ts.index[0], '%Y-%m-%d'), season='monthly', deltat='1/12 year', quiet=True, print_progress=False)
rb.plot(oc)


INFO: '2014-01-31' interpreted as 2014-01-01 (Y-M-D)



(<Figure size 640x480 with 9 Axes>,
 array([<Axes: xlabel='[]', ylabel='Y'>,
        <Axes: xlabel='[]', ylabel='season'>,
        <Axes: xlabel='[]', ylabel='Pr(scp)'>,
        <Axes: xlabel='[]', ylabel='sOrder'>,
        <Axes: xlabel='[]', ylabel='trend'>,
        <Axes: xlabel='[]', ylabel='Pr(tcp)'>,
        <Axes: xlabel='[]', ylabel='tOrder'>,
        <Axes: xlabel='[]', ylabel='slpsgn'>,
        <Axes: xlabel='time', ylabel='error'>], dtype=object))

In [19]:
max(o.season.Y) - min(o.season.Y)

0.09396128

In [16]:
import pandas as pd
d = sea_level.sel(latitude=0.5, longitude=73.1, method='nearest').time[0].values
# transform d from numpy.datetime64 to datetime
d = pd.to_datetime(d)

o = rb.beast(sea_level.sel(latitude=0.5, longitude=73.1, method='nearest').values, start = [d.year, d.month, d.day], season='monthly', deltat='1/12 year', quiet=True, print_progress=False)
rb.plot(o)

# plt.plot(sea_level.sel(latitude=0.5, longitude=73.1, method='nearest').time, o.season.Y)

(<Figure size 640x480 with 9 Axes>,
 array([<Axes: xlabel='[]', ylabel='Y'>,
        <Axes: xlabel='[]', ylabel='season'>,
        <Axes: xlabel='[]', ylabel='Pr(scp)'>,
        <Axes: xlabel='[]', ylabel='sOrder'>,
        <Axes: xlabel='[]', ylabel='trend'>,
        <Axes: xlabel='[]', ylabel='Pr(tcp)'>,
        <Axes: xlabel='[]', ylabel='tOrder'>,
        <Axes: xlabel='[]', ylabel='slpsgn'>,
        <Axes: xlabel='time', ylabel='error'>], dtype=object))

In [53]:
# function to normalise between 0 and 1
def normalise(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

plt.figure()
plt.plot(sea_level.sel(latitude=0.5, longitude=73.1, method='nearest').time, normalise(o.season.Y))
plt.plot(wave_energy.sel(latitude=0.5, longitude=73.1, method='nearest').time, normalise(ow.season.Y))
plt.xlim(datetime(2013, 1, 1), datetime(2022, 12, 31))

(15706.0, 19357.0)

In [15]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

# Define the values for idxx and idxy
idxx_values = [0, 1, 2]
idxy_values = [0, 1, 2]

# Create a figure and axes for the joint plot
fig, axs = plt.subplots(3, 3, figsize=(20, 10))
# fig2, axs2 = plt.subplots(3, 3, subplot_kw={'projection': 'polar'}, figsize=(20, 10))

# Choose a common colormap for both panels
cmap = plt.get_cmap('plasma')

# Normalize wave energy values to range [0, 1]
global_min_wave_energy = float('inf')
global_max_wave_energy = float('-inf')

# Iterate over idxx and idxy values to create the subplots
for idxx in idxx_values:
    for idxy in idxy_values:
        # Sample data for wave direction and energy for NE monsoon (January, February, March, December)
        we = wave_energy.isel(latitude=idxy, longitude=idxx)
        lat = we.latitude.values
        lon = we.longitude.values
        sl = sea_level.sel(latitude=lat, longitude=lon, method='nearest').values.flatten() # Assuming wave_direction is already in degrees

        axs[idxx, idxy].plot(wave_energy.isel(latitude=idxy, longitude=idxx).time, we.values.flatten(), label='Wave Energy', color='blue')
        axs[idxx, idxy].plot(sea_level.isel(latitude=idxy, longitude=idxx).time, sl, label='Sea Level', color='red')

# Add a common color bar for wave energy
# norm = mcolors.Normalize(vmin=global_min_wave_energy, vmax=global_max_wave_energy)
# cbar = fig.colorbar(plt.cm.ScalarMappable(cmap=cmap, norm=norm), ax=axs, orientation='vertical')
# cbar.set_label('Wave Energy (kW/m)', fontsize=20)

# cbar2 = fig.colorbar(plt.cm.ScalarMappable(cmap=cmap, norm=norm), ax=axs2, orientation='vertical', label='Wave Energy')
# cbar2.set_label('Wave Energy (kW/m)', fontsize=20)

# fig.suptitle(r'Northeast Monsoon [December$-$February]', fontweight='bold', fontsize=20)
# fig2.suptitle(r'Southwest Monsoon [May$-$September]', fontweight='bold', fontsize=20)

# Adjust layout
#plt.tight_layout()

plt.show()

NameError: name 'we' is not defined

In [16]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

figg, axx = plt.subplots()
huvadhu_atoll.plot(ax=axx, facecolor='none', edgecolor='black', linewidth=3)
axx.axis('off')

# Define the values for idxx and idxy
idxx_values = [0, 1, 2]
idxy_values = [0, 1, 2]

# Create a figure and axes for the joint plot
fig, axs = plt.subplots(3, 3, subplot_kw={'projection': 'polar'}, figsize=(20, 10))
fig2, axs2 = plt.subplots(3, 3, subplot_kw={'projection': 'polar'}, figsize=(20, 10))

# Choose a common colormap for both panels
cmap = plt.get_cmap('plasma')

# Normalize wave energy values to range [0, 1]
global_min_wave_energy = float('inf')
global_max_wave_energy = float('-inf')

# Iterate over idxx and idxy values to create the subplots
for idxx in idxx_values:
    for idxy in idxy_values:
        axx.plot(clip_data_waves['wave_energy'].isel(latitude=idxy, longitude=idxx).sel(time=clip_data_waves['wave_energy'].time.dt.month.isin([1, 2, 12])).longitude, clip_data_waves['wave_energy'].isel(latitude=idxy, longitude=idxx).sel(time=clip_data_waves['wave_energy'].time.dt.month.isin([1, 2, 3, 12])).latitude, 'ro')
        fig3, ax3 = plt.subplots(1, 1, subplot_kw={'projection': 'polar'})
        fig4, ax4 = plt.subplots(1, 1, subplot_kw={'projection': 'polar'})
        fig5, ax5 = plt.subplots(1, 1, subplot_kw={'projection': 'polar'})
        fig6, ax6 = plt.subplots(1, 1, subplot_kw={'projection': 'polar'})
        # Sample data for wave direction and energy for NE monsoon (January, February, March, December)
        wave_energy_ne_monsoon = clip_data_waves['wave_energy'].isel(latitude=idxy, longitude=idxx).sel(time=clip_data_waves['wave_energy'].time.dt.month.isin([1, 2, 12])).values.flatten()
        wave_direction_ne_monsoon = clip_data_waves['mwd'].isel(latitude=idxy, longitude=idxx).sel(time=clip_data_waves['wave_energy'].time.dt.month.isin([1, 2, 12])).values.flatten() # Assuming wave_direction is already in degrees

        # Sample data for wave direction and energy for SW monsoon (April to November)
        wave_energy_sw_monsoon = clip_data_waves['wave_energy'].isel(latitude=idxy, longitude=idxx).sel(time=clip_data_waves['wave_energy'].time.dt.month.isin([5, 6, 7, 8, 9])).values.flatten()
        wave_direction_sw_monsoon = clip_data_waves['mwd'].isel(latitude=idxy, longitude=idxx).sel(time=clip_data_waves['wave_energy'].time.dt.month.isin([5, 6, 7, 8, 9])).values.flatten() # Assuming wave_direction is already in degrees

        # Sample data for wave direction and energy for transition NE to SW monsoon (March, April)
        wave_energy_transition_ne_sw = clip_data_waves['wave_energy'].isel(latitude=idxy, longitude=idxx).sel(time=clip_data_waves['wave_energy'].time.dt.month.isin([3, 4])).values.flatten()
        wave_direction_transition_ne_sw = clip_data_waves['mwd'].isel(latitude=idxy, longitude=idxx).sel(time=clip_data_waves['wave_energy'].time.dt.month.isin([3, 4])).values.flatten() # Assuming wave_direction is already in degrees

        # Sample data for wave direction and energy for transition SW to NE monsoon (October, November)
        wave_energy_transition_sw_ne = clip_data_waves['wave_energy'].isel(latitude=idxy, longitude=idxx).sel(time=clip_data_waves['wave_energy'].time.dt.month.isin([10, 11])).values.flatten()
        wave_direction_transition_sw_ne = clip_data_waves['mwd'].isel(latitude=idxy, longitude=idxx).sel(time=clip_data_waves['wave_energy'].time.dt.month.isin([10, 11])).values.flatten() # Assuming wave_direction is already in degrees

        # Combine wave_energy and wave_direction for both monsoon seasons
        all_wave_energy = np.concatenate((wave_energy_ne_monsoon, wave_energy_sw_monsoon, wave_energy_transition_ne_sw, wave_energy_transition_sw_ne))
        all_wave_direction = np.concatenate((wave_direction_ne_monsoon, wave_direction_sw_monsoon, wave_direction_transition_ne_sw, wave_direction_transition_sw_ne))

        # Calculate global min and max for all_wave_energy
        local_min_wave_energy = np.min(all_wave_energy)
        local_max_wave_energy = np.max(all_wave_energy)
        if local_min_wave_energy < global_min_wave_energy:
            global_min_wave_energy = local_min_wave_energy
        if local_max_wave_energy > global_max_wave_energy:
            global_max_wave_energy = local_max_wave_energy

        # Plot wave energy using a colormap
        ax = axs[idxy, idxx]
        ax2 = axs2[idxy, idxx]
        bars = ax.bar(np.radians(wave_direction_ne_monsoon), wave_energy_ne_monsoon, width=0.1, color=cmap((wave_energy_ne_monsoon - global_min_wave_energy) / (global_max_wave_energy - global_min_wave_energy)), alpha=0.7, label='Wave Energy NE Monsoon', zorder=5)
        bars2 = ax2.bar(np.radians(wave_direction_sw_monsoon), wave_energy_sw_monsoon, width=0.1, color=cmap((wave_energy_sw_monsoon - global_min_wave_energy) / (global_max_wave_energy - global_min_wave_energy)), alpha=0.7, label='Wave Energy SW Monsoon', zorder=5)

        bars3 = ax3.bar(np.radians(wave_direction_ne_monsoon), wave_energy_ne_monsoon, width=0.1, color=cmap((wave_energy_ne_monsoon - global_min_wave_energy) / (global_max_wave_energy - global_min_wave_energy)), alpha=0.7, label='Wave Energy NE Monsoon', zorder=5)
        bars4 = ax4.bar(np.radians(wave_direction_sw_monsoon), wave_energy_sw_monsoon, width=0.1, color=cmap((wave_energy_sw_monsoon - global_min_wave_energy) / (global_max_wave_energy - global_min_wave_energy)), alpha=0.7, label='Wave Energy SW Monsoon', zorder=5)
        
        bars5 = ax5.bar(np.radians(wave_direction_transition_ne_sw), wave_energy_transition_ne_sw, width=0.1, color=cmap((wave_energy_transition_ne_sw - global_min_wave_energy) / (global_max_wave_energy - global_min_wave_energy)), alpha=0.7, label='Wave Energy Transition NE to SW Monsoon', zorder=5)
        bars6 = ax6.bar(np.radians(wave_direction_transition_sw_ne), wave_energy_transition_sw_ne, width=0.1, color=cmap((wave_energy_transition_sw_ne - global_min_wave_energy) / (global_max_wave_energy - global_min_wave_energy)), alpha=0.7, label='Wave Energy Transition SW to NE Monsoon', zorder=5)

        
        # Set the theta zero location and direction
        # Set the theta zero location and direction
        ax.set_theta_zero_location('N')
        ax.set_theta_direction(-1)
        ax2.set_theta_zero_location('N')
        ax2.set_theta_direction(-1)
        ax3.set_theta_zero_location('N')
        ax3.set_theta_direction(-1)
        ax4.set_theta_zero_location('N')
        ax4.set_theta_direction(-1)
        ax5.set_theta_zero_location('N')
        ax5.set_theta_direction(-1)

        ax.tick_params(axis='x', which='major', labelsize=12, width=2, direction='out', pad=0.5)
        ax2.tick_params(axis='x', which='major', labelsize=12, width=2, direction='out', pad=0.5)
        ax3.tick_params(axis='x', which='major', labelsize=12, width=2, direction='out', pad=0.5)
        ax4.tick_params(axis='x', which='major', labelsize=12, width=2, direction='out', pad=0.5)
        ax5.tick_params(axis='x', which='major', labelsize=12, width=2, direction='out', pad=0.5)
        ax6.tick_params(axis='x', which='major', labelsize=12, width=2, direction='out', pad=0.5)

        ax.tick_params(axis='y', which='major', width=2, direction='out', pad=1, color='gray', zorder=15, grid_zorder=1)
        ax2.tick_params(axis='y', which='major', width=2, direction='out', pad=1, color='gray', zorder=15, grid_zorder=1)
        ax3.tick_params(axis='y', which='major', width=2, direction='out', pad=1, color='gray', zorder=15, grid_zorder=1) 
        ax4.tick_params(axis='y', which='major', width=2, direction='out', pad=1, color='gray', zorder=15, grid_zorder=1)

        fig3.savefig(f'figures//process regime//NE_wave_energy_{idxx}_{idxy}.png', transparent=True)  
        fig4.savefig(f'figures//process regime//SW_wave_energy_{idxx}_{idxy}.png', transparent=True)

# Add a common color bar for wave energy
norm = mcolors.Normalize(vmin=global_min_wave_energy, vmax=global_max_wave_energy)
cbar = fig.colorbar(plt.cm.ScalarMappable(cmap=cmap, norm=norm), ax=axs, orientation='vertical')
cbar.set_label('Wave Energy (kW/m)', fontsize=20)

cbar2 = fig.colorbar(plt.cm.ScalarMappable(cmap=cmap, norm=norm), ax=axs2, orientation='vertical', label='Wave Energy')
cbar2.set_label('Wave Energy (kW/m)', fontsize=20)

fig.suptitle(r'Northeast Monsoon [December$-$February]', fontweight='bold', fontsize=20)
fig2.suptitle(r'Southwest Monsoon [May$-$September]', fontweight='bold', fontsize=20)

# Adjust layout
#plt.tight_layout()

plt.show()

In [152]:

for month in range(1, 13):

    fig, ax = plt.subplots()

    clip_data_waves = data_ERA5_waves['mwd'].sel(time='2013-{}-01'.format(month), latitude=slice(1.50, 0.), longitude=slice(72.6, 74.))
    clip_data_energy = data_ERA5_waves['swh'].sel(time='2013-{}-01'.format(month), latitude=slice(1.50, 0.), longitude=slice(72.6, 74.))
    clip_data_energy.plot(ax=ax)

    # Overlaying the area boundary
    area.plot(ax=ax, facecolor='none', edgecolor='black')

    # Plotting wind vectors
    for i in range(len(clip_data_waves['latitude'])):
        for j in range(len(clip_data_waves['longitude'])):
            latitude_value = clip_data_waves['latitude'].values[i]
            longitude_value = clip_data_waves['longitude'].values[j]
            value = clip_data_waves.sel(latitude=latitude_value, longitude=longitude_value).values
            
            # Plotting wind vectors using quiver
            ax.quiver(longitude_value, latitude_value, 0.2*np.cos(np.radians(value)), 0.2*np.sin(np.radians(value)),
                    angles='xy', scale_units='xy', scale=1)
    
    plt.title('Month: {}'.format(month))

    # Show the plot
    plt.show()

In [144]:
import numpy as np
import matplotlib.pyplot as plt
import calendar

# Assuming data_ERA5_waves['mwd'] contains data for each month of the year

# Iterate over each month
for month_num in range(1, 13):
    # Extract angles for the current month
    angles = data_ERA5_waves['mwd'].sel(latitude=slice(1.50, 0.), longitude=slice(72.6, 74.), time=data_ERA5_waves['mwd'].time.dt.month == month_num).values[0]

    # Number of bins (directions)
    num_bins = 16

    # Convert angles to radians
    angles = np.radians(angles)

    # Create the histogram
    hist, bins = np.histogram(angles, bins=num_bins, range=(0, 2*np.pi))

    # Compute width of each bin
    width = 2 * np.pi / num_bins

    # Create the rose diagram
    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
    bars = ax.bar(bins[:-1], hist, width=width, color='skyblue', edgecolor='black')

    # Add labels to the bars
    ax.set_theta_zero_location('N')
    ax.set_theta_direction(-1)
    ax.set_xticks(np.arange(0, 2*np.pi, np.pi/4))
    ax.set_xticklabels(['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW'])

    plt.title(f'Rose Diagram for {calendar.month_name[month_num]}')
    plt.show()

In [137]:
angles = data_ERA5_waves['mwd'].sel(latitude=slice(1.50, 0.), longitude=slice(72.6, 74.)).values.flatten()

# Number of bins (directions)
num_bins = 16

# Convert angles to radians
angles = np.radians(angles)

# Create the histogram
hist, bins = np.histogram(angles, bins=num_bins, range=(0, 2*np.pi))

# Compute width of each bin
width = 2 * np.pi / num_bins

# Create the rose diagram
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
bars = ax.bar(bins[:-1], hist, width=width, color='skyblue', edgecolor='black')

# Add labels to the bars
ax.set_theta_zero_location('N')
ax.set_theta_direction(-1)
ax.set_xticks(np.arange(0, 2*np.pi, np.pi/4))
ax.set_xticklabels(['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW'])

plt.title('Rose Diagram')
plt.show()

In [107]:
fig, ax = plt.subplots()
data_ERA5_waves['mwd'].sel(time='2015-01-01', latitude=slice(1.50, 0.), longitude=slice(72.6, 74.)).plot()
ax.quiver(72.853, 1.098, 0.2*np.cos(np.radians(120)), 0.2*np.sin(np.radians(120)), angles='xy', scale_units='xy', scale=1)
area.plot(ax=ax, facecolor='none', edgecolor='black')

<Axes: title={'center': 'time = 2015-01-01'}, xlabel='longitude [degrees_east]', ylabel='latitude [degrees_north]'>

# Plot by region

In [239]:
island_info = retrieve_island_info('Dhakandhoo', 'Maldives', verbose=False)

plot_shoreline_transects(island_info)

transect = 55

sc = island_info['timeseries_preprocessing']['optimal time period']['dict_timeseries']['coastline_position_transect_{}_waterline'.format(transect)]['monthly']

wave_energy = sc['wave_energy_of_combined_wind_waves_and_swell']
sea_level = sc['sea_level_anomaly']
sst = sc['sea_surface_temperature']
wave_direction = sc['mean_wave_direction']
coastline = sc['coastline_position_transect_{}_waterline'.format(transect)]
wind = sc['wind_speed_10m']

In [246]:
sc.keys()

Index(['coastline_position_transect_55_waterline',
       'sea_surface_temperature_NOAACRW', 'sea_level_anomaly',
       '2_metre_dewpoint_temperature', 'soil_temperature_level_1',
       'total_precipitation', 'evaporation', 'sea_surface_temperature',
       'mean_sea_level_pressure', 'mean_direction_of_total_swell',
       'mean_direction_of_wind_waves', 'mean_wave_direction', 'wind_speed_10m',
       'wind_direction_10m', 'wind_direction_true_10m',
       'wave_energy_of_total_swell', 'wave_energy_of_wind_waves',
       'wave_energy_of_combined_wind_waves_and_swell'],
      dtype='object')

In [62]:
# Min-Max Scaling
def min_max_scaling(series):
    return (series - series.min()) / (series.max() - series.min())

In [149]:
fig, ax = plt.subplots()
ax.plot(wave_energy.index, min_max_scaling(wave_energy.values))
ax.plot(sea_level.index, min_max_scaling(sea_level.values))
ax.plot(sst.index, min_max_scaling(sst.values))
ax.plot(wave_direction.index, min_max_scaling(wave_direction.values))
ax.plot(coastline.index, min_max_scaling(coastline.values))

In [231]:
from scipy import signal

list_var = [wave_energy, sea_level, sst, coastline]
list_var = [wave_energy, coastline]
#list_var = [sea_level, coastline]
fig, ax = plt.subplots()
o_cp = rb.beast(coastline.values, start=[coastline.index[0].year, coastline.index[0].month, coastline.index[0].day], period='1 year', deltat='1/12 year', quiet=True, print_progress=False)
for var in list_var:
    o = rb.beast(var.values, start=[var.index[0].year, var.index[0].month, var.index[0].day], period='1 year', deltat='1/12 year', quiet=True, print_progress=False)
    rb.plot(o)
    fig2, ax2 = plt.subplots()
    ax.plot(var.index, min_max_scaling(o.season.Y))
    #ax2.scatter(coastline.values, var.values)
    #ax2.scatter(o_cp.season.Y, o.season.Y)
    y = o_cp.season.Y
    x = o.season.Y
    correlation = signal.correlate(x-np.mean(x), y-np.mean(y), mode="full")
    lags = signal.correlation_lags(len(x), len(y), mode="full")
    lag = lags[np.argmax(abs(correlation))]
    corr = correlation[np.argmax(abs(correlation))]
    #ax2.set_title(np.corrcoef(o_cp.season.Y, o.season.Y)[0, 1])
    #ax2.set_title('lag: {}, corr: {}'.format(lag, corr))
    ax2.plot(lags, correlation)
    ax2.set_title(lag)
    
    break




In [203]:
plt.plot(np.arange(-len(x) + 1, len(x)), abs(np.correlate(x-np.mean(x), y-np.mean(y), mode='full')))

In [183]:
lags[np.argsort(abs(correlation))][::-1]

array([ -7,   4,  -8,   5, -19,  16, -20,  17, -31,  -1,  -2, -13, -25,
        28, -14, -32,   0,  10,  11, -24, -12, -18,   3,  -6, -43,  29,
       -26, -37, -30,  -3,  40, -44,   6,  -9,  15, -36,  12,   9,  22,
       -38,  23, -55, -42, -49,  41, -15,  27,  21,  52,  18, -56,  24,
       -48, -50, -21,  34, -23,  39, -11,  35,  33, -27, -54, -61, -33,
       -67,   1,  53, -35,  51,  64,  30, -39, -60,  36, -62,  45, -68,
        46, -45,  13,   8,  63, -29,  47, -51,  -4, -47, -66,  57,  20,
        32, -17, -57, -73,  44,  56,  25,  76,  75,  68, -41,  42,  65,
        58, -74,  48, -79, -72, -63,  69, -59, -80, -69,  79,  80, -53,
        37,  59, -78,  67,  -5,  87, -16, -75, -71,  70,  74,  88, -85,
        62,  54,  81, -65,   2,  50,  60, -81, -86, -84,  38,  55,  14,
        26,   7,  49,  77,  91,  78,  90, -91, -92, -87, -83, -28,  86,
       -40, -77, -52, -90,  19, -64,  92,  71,  43, -76,  95,  82, -93,
        73,  61, -22, -88,  93, -10,  94,  96,  83, -89,  66, -3

In [169]:
plt.plot(lags, abs(correlation))

In [170]:
np.corrcoef(o_cp.season.Y[4:], o.season.Y[:-4])[0, 1]

-0.9200372746666252

In [189]:
plt.plot(o.season.Y)
plt.plot(o_cp.season.Y[4:])

In [187]:
plt.plot(o.season.Y[:-4])
plt.plot(o_cp.season.Y[4:])

In [232]:
from scipy.signal import find_peaks, argrelextrema

find_peaks(o_cp.season.Y)

plt.plot(coastline.index, o_cp.season.Y, color='g')
plt.plot(coastline.index, o.season.Y, color='r')

for peak in argrelextrema(o_cp.season.Y, np.less)[0]:
    plt.axvline(coastline.index[peak], color='g', ls=':')

for peak in find_peaks(o.season.Y)[0]:
    plt.axvline(coastline.index[peak], color='r', ls=':')

In [243]:
fig, ax = plt.subplots()
time_series = coastline

# Plot fitting sinusoidal function
#for peak in find_peaks(o_cp.season.Y)[0]:
    #ax.axvline(time_series.index[peak], color='r', linestyle='--')
#for minimum in argrelextrema(o_cp.season.Y, np.less)[0]:
    #ax.axvline(time_series.index[minimum], color='pink', linestyle='--')
for year in range(2010, 2023):
    if year == 2010:
        ax.axvspan(datetime.datetime(year=year, month=12, day=1), datetime.datetime(year=year+1, month=2, day=28), color='yellow', alpha=0.2, label='NE Monsoon')
        ax.axvspan(datetime.datetime(year=year, month=5, day=1), datetime.datetime(year=year, month=9, day=30), color='blue', alpha=0.2, label='SW Monsoon')
    else:
        ax.axvspan(datetime.datetime(year=year, month=12, day=1), datetime.datetime(year=year+1, month=2, day=28), color='yellow', alpha=0.2)
        ax.axvspan(datetime.datetime(year=year, month=5, day=1), datetime.datetime(year=year, month=9, day=30), color='blue', alpha=0.2)
ax.plot(time_series.index, min_max_scaling(o_cp.season.Y), color='g')
var = wave_energy
o = rb.beast(var.values, start=[var.index[0].year, var.index[0].month, var.index[0].day], period='1 year', deltat='1/12 year', quiet=True, print_progress=False)
ax.plot(time_series.index, min_max_scaling(o.season.Y), color='black')
var = wave_direction
o = rb.beast(var.values, start=[var.index[0].year, var.index[0].month, var.index[0].day], period='1 year', deltat='1/12 year', quiet=True, print_progress=False)
ax.plot(time_series.index, min_max_scaling(o.season.Y), color='gray')
ax.legend(fontsize=10)
ax.set_xlim(time_series.index[0], time_series.index[-1])
#ax.set_title('Name: {}'.format(time_series_name))
plt.show()

In [251]:
data_ERA5_waves['mwd'].values

array([[[147.4288  ,  97.38376 ,  92.409966, 102.785286],
        [139.57602 ,  92.38803 ,  85.64845 ,  97.47698 ],
        [143.135   ,  91.532555,  77.65858 ,  91.614815],
        ...,
        [157.7109  , 139.55408 , 120.89824 , 127.53362 ],
        [159.75087 , 155.819   , 141.79147 , 136.14317 ],
        [161.19312 , 159.92087 , 150.13779 , 143.52983 ]],

       [[120.75566 ,  64.78264 ,  68.66516 ,  86.273605],
        [122.455635,  72.80542 ,  70.754486,  86.57521 ],
        [131.53679 ,  76.5673  ,  67.79872 ,  85.55523 ],
        ...,
        [145.08174 , 127.670715, 108.7352  , 113.648674],
        [150.01715 , 143.45854 , 127.66524 , 123.700455],
        [153.33484 , 151.41551 , 139.97086 , 133.52191 ]],

       [[194.9184  , 156.66899 , 129.48586 , 132.98999 ],
        [191.17297 , 151.31133 , 122.16499 , 128.76747 ],
        [190.37782 , 145.88786 , 113.84061 , 124.15012 ],
        ...,
        [176.28448 , 172.561   , 165.8324  , 164.81241 ],
        [175.8403  , 175.6374

In [247]:
fig, ax = plt.subplots()
time_series = coastline

# Plot fitting sinusoidal function
#for peak in find_peaks(o_cp.season.Y)[0]:
    #ax.axvline(time_series.index[peak], color='r', linestyle='--')
#for minimum in argrelextrema(o_cp.season.Y, np.less)[0]:
    #ax.axvline(time_series.index[minimum], color='pink', linestyle='--')
for year in range(2010, 2023):
    if year == 2010:
        ax.axvspan(datetime.datetime(year=year, month=12, day=1), datetime.datetime(year=year+1, month=2, day=28), color='yellow', alpha=0.2, label='NE Monsoon')
        ax.axvspan(datetime.datetime(year=year, month=5, day=1), datetime.datetime(year=year, month=9, day=30), color='blue', alpha=0.2, label='SW Monsoon')
    else:
        ax.axvspan(datetime.datetime(year=year, month=12, day=1), datetime.datetime(year=year+1, month=2, day=28), color='yellow', alpha=0.2)
        ax.axvspan(datetime.datetime(year=year, month=5, day=1), datetime.datetime(year=year, month=9, day=30), color='blue', alpha=0.2)
var = wave_direction
o = rb.beast(var.values, start=[var.index[0].year, var.index[0].month, var.index[0].day], period='1 year', deltat='1/12 year', quiet=True, print_progress=False)
ax.plot(time_series.index, o.season.Y, color='gray')
ax.legend(fontsize=10)
ax.set_xlim(time_series.index[0], time_series.index[-1])
#ax.set_title('Name: {}'.format(time_series_name))
plt.show()

# Test for asymmetric sinusoidal data

In [120]:
def asymmetric_sinusoidal(x, A, k, period1, period2, phi, offset):
    return A * np.sin(2 * np.pi / period1 * x + k * np.sin(2 * np.pi / period2 * x) + phi) + offset

In [114]:
x = np.linspace(0, 4*np.pi, 100)
y = np.sin(x)
y2 = np.sin(x + 0.5*np.sin(x)) 

# add random noise
y += np.random.normal(0, 0.1, y.shape)

# add trend
y += 0.1 * x

fig, ax = plt.subplots()
ax.plot(x, y)
ax.plot(x, y2)

# find peaks

from scipy.signal import find_peaks

peaks, _ = find_peaks(y2)
ax.plot(x[peaks], y2[peaks], "x")

# find minima
from scipy.signal import argrelextrema

minima = argrelextrema(y2, np.less)
ax.plot(x[minima], y2[minima], "o")
plt.show()

In [115]:
import Rbeast as rb

o = rb.beast(y2, start=[0], quiet=True, print_progress=False)
rb.plot(o)

(<Figure size 640x480 with 9 Axes>,
 array([<Axes: xlabel='[]', ylabel='Y'>,
        <Axes: xlabel='[]', ylabel='season'>,
        <Axes: xlabel='[]', ylabel='Pr(scp)'>,
        <Axes: xlabel='[]', ylabel='sOrder'>,
        <Axes: xlabel='[]', ylabel='trend'>,
        <Axes: xlabel='[]', ylabel='Pr(tcp)'>,
        <Axes: xlabel='[]', ylabel='tOrder'>,
        <Axes: xlabel='[]', ylabel='slpsgn'>,
        <Axes: xlabel='time', ylabel='error'>], dtype=object))

In [107]:
x[peaks][1] - x[minima][0]

2.1578616206475347

In [131]:
# def f(x, delta=0.5):

#     o1 = np.pi + 0.5*np.pi*delta
#     o2 = np.pi - 0.5*np.pi*(1 - delta)

#     print(o1, o2)
    
#     res = []

#     for i in range(len(x)):
#         if x[i] < o1:
#             res.append(-1*np.cos(x[i] * np.pi / o1))
#         else:
#             res.append(-1*np.cos(x[i] * np.pi / o2))
#     return res

def f(x, delta=0.5):
    o1 = np.pi + 0.5 * np.pi * delta
    o2 = np.pi - 0.5 * np.pi * (1 - delta)
    T = o1 + o2  # Total period

    res = []
    for xi in x:
        xi_mod = xi % T  # Make the function periodic
        if xi_mod < o1:
            res.append(-1 * np.cos(xi_mod * np.pi / o1))
        else:
            res.append(-1 * np.cos((xi_mod - o1) * np.pi / o2))
    return res

x = np.linspace(0, 2*np.pi, 1000, endpoint=True)
plt.plot(x, )

In [139]:
import math

def asymsin(t, f, delta):
    # Define the asymmetric sinusoidal function
    omega1 = math.pi + math.pi/2 * delta
    omega2 = math.pi - math.pi/2 * (1 - delta)
    
    if t < omega1:
        return -math.cos(t * math.pi / omega1)
    else:
        return -math.cos(t * math.pi / omega2)

def compute_vibration(A, alpha, t, f, delta):
    # Compute vibrations along x and y axes
    x = A * math.cos(alpha) * asymsin(t * f, f, delta)
    y = A * math.sin(alpha) * asymsin(t * f, f, delta)
    return x, y

# Example usage:
A = 1.0        # Amplitude
alpha = math.pi / 4  # Angle in radians (45 degrees)
t = 0.5        # Time in seconds
f = 1.0        # Frequency in Hertz
delta = 0.5    # Asymmetry parameter

# Compute vibrations
x, y = compute_vibration(A, alpha, t, f, delta)

# Print results
print(f"Vibration along x-axis: {x}")
print(f"Vibration along y-axis: {y}")

# plot 
import matplotlib.pyplot as plt

# Define time range
t = np.linspace(0, 2 * np.pi, 1000)

# Compute vibrations along x and y axes
x = [compute_vibration(A, alpha, ti, f, delta)[0] for ti in t]
y = [compute_vibration(A, alpha, ti, f, delta)[1] for ti in t]

# Plot the vibrations
plt.plot(t, x, label='Vibration along x-axis')
plt.plot(t, y, label='Vibration along y-axis')

Vibration along x-axis: -0.651288474745862
Vibration along y-axis: -0.651288474745862


In [143]:
delta = 0.5
omega1 = np.pi + np.pi/2 * delta
omega2 = np.pi - np.pi/2 * (1 - delta)

x = np.linspace(0, 2*np.pi, 1000)

y1 = np.where(x < omega1, -np.cos(x * np.pi / omega1), -np.cos(x * np.pi / omega2))
y2 = np.where(x < omega1, -np.cos(x * np.pi / omega1), -np.cos((x - omega1) * np.pi / omega2))



In [144]:

import numpy as np
import matplotlib.pyplot as plt

def asymmetric_sin_wave(t, T1, T2, omega1, omega2, A):
    """
    Generate an asymmetric sinusoidal waveform.
    
    Parameters:
    t: array-like
        Time array.
    T1: float
        Duration of the shorter half period.
    T2: float
        Duration of the longer half period.
    omega1: float
        Angular frequency for the shorter half period.
    omega2: float
        Angular frequency for the longer half period.
    A: float
        Amplitude of the sinusoidal signal.
    
    Returns:
    x: array-like
        Asymmetric sinusoidal waveform.
    """
    x = np.zeros_like(t)
    x[(t >= 0) & (t < T1)] = A * np.sin(omega1 * t[(t >= 0) & (t < T1)])
    x[(t >= T1) & (t < T1 + T2)] = A * np.sin(omega2 * t[(t >= T1) & (t < T1 + T2)])
    return x

# Parameters
T1 = 1.0   # Duration of the shorter half period
T2 = 2.0   # Duration of the longer half period
omega1 = 2 * np.pi / T1   # Angular frequency for the shorter half period
omega2 = 2 * np.pi / T2   # Angular frequency for the longer half period
A = 1.0    # Amplitude

# Time array
t = np.linspace(0, T1 + T2, num=1000)

# Generate asymmetric sinusoidal waveform
x = asymmetric_sin_wave(t, T1, T2, omega1, omega2, A)

# Plotting
plt.figure(figsize=(10, 5))
plt.plot(t, x, label='Asymmetric Sinusoidal Waveform')
plt.title('Asymmetric Sinusoidal Waveform')
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.grid(True)
plt.legend()
plt.show()


In [145]:
import numpy as np
import matplotlib.pyplot as plt

# Define parameters
T = 2 * np.pi  # Period of the wave
T1 = T / 2     # Point where wave shape changes

# Define the asymmetric wave function
def asymmetric_wave(t):
    if 0 <= t < T1:
        return np.sin(t)
    elif T1 <= t < T:
        return np.sign(np.sin(t))

# Generate time values for plotting
t_values = np.linspace(0, T, 1000)

# Calculate the wave values
wave_values = np.array([asymmetric_wave(t) for t in t_values])

# Plotting
plt.figure(figsize=(8, 4))
plt.plot(t_values, wave_values, label='Asymmetric Wave')
plt.title('Asymmetric Waveform Example')
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
def titled_sin(x, n):

    y = np.sin(x)

In [158]:
import numpy as np
import matplotlib.pyplot as plt

# Define the parameters
n = 1  # Parameter affecting the waveform
x = np.linspace(0, 10*np.pi, 1000)  # x values from 0 to 10*pi

# Initialize y as zeros (or initial guess)
y = np.zeros_like(x)

# Define the tolerance and maximum number of iterations
tolerance = 1e-6
max_iterations = 100

# Iterate to find y using the implicit equation y = sin(x + y * n)
for i in range(max_iterations):
    y_new = np.sin(x + y * n)
    error = np.max(np.abs(y_new - y))
    y = y_new
    
    if error < tolerance:
        print(f'Converged after {i+1} iterations with error {error:.6e}')
        break

# Plot the waveform
plt.figure(figsize=(8, 4))
plt.plot(x, y, label='y = sin(x + y*n)')
plt.title('Implicit Waveform y = sin(x + y*n)')
plt.xlabel('x')
plt.ylabel('y')
plt.grid(True)
plt.legend()
plt.show()


In [165]:
x = np.linspace(0, 100, 1000)
t = np.arange(0, 1, 0.1)
# y = 1/t * np.arctan((t * np.sin(x)/(1 - t*np.cos(x))))

for i in t:
    plt.plot(x, 1/i * np.arctan((i * np.sin(x)/(1 - i*np.cos(x)))))
# plt.plot(x, y)

In [56]:
island = "Hulhudhoo (South Maalhosmadulu)"

island_info = retrieve_island_info(island, 'Maldives', verbose=False)

ts_name = 'coastline_position_transect_10_waterline'
ts = island_info['timeseries_preprocessing']['optimal time period']['dict_timeseries'][ts_name]['monthly'][ts_name] 
p = island_info['timeseries_analysis'][ts_name]['seasonality']['peaks_seasonal_BEAST']
# stats.mode([ts.index[p][i].month for i in range(len(p))]).mode

In [178]:
type(params[3])

IndexError: index 3 is out of bounds for axis 0 with size 3

In [272]:
rb.plot(o)

(<Figure size 640x480 with 9 Axes>,
 array([<Axes: xlabel='[]', ylabel='Y'>,
        <Axes: xlabel='[]', ylabel='season'>,
        <Axes: xlabel='[]', ylabel='Pr(scp)'>,
        <Axes: xlabel='[]', ylabel='sOrder'>,
        <Axes: xlabel='[]', ylabel='trend'>,
        <Axes: xlabel='[]', ylabel='Pr(tcp)'>,
        <Axes: xlabel='[]', ylabel='tOrder'>,
        <Axes: xlabel='[]', ylabel='slpsgn'>,
        <Axes: xlabel='time', ylabel='error'>], dtype=object))

In [115]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Define the Clausen function
# def clausen(x, *params):
#     result = np.zeros_like(x, dtype=np.float64)
#     for k, param in enumerate(params, start=1):
#         result += np.sin(k * x) / k**2
#     return result

def clausen(x, A, B):
    result = A * np.sum(np.sin(np.outer(np.arange(1, len(x)+1), x)) / np.arange(1, len(x)+1)**2, axis=0) + B
    return result

# def seasonal_function_fitting(x, A, k, period, phi, offset):
#     return A * np.exp(k * x) * np.sin(2 * np.pi / period * x + phi) + offset

def seasonal_function_fitting(x, A, k, sp, period, phi, offset):
    inside_sin = 2 * np.pi / period * x + phi
    skewed_sin = np.sin(inside_sin) / np.sqrt((sp + np.cos(inside_sin))**2 + np.sin(inside_sin)**2)
    return A * np.exp(k * x) * skewed_sin + offset

# Generate some synthetic data
np.random.seed(0)
x_data = np.array([(date - ts.index[0]).days for date in ts.index])
x_data_big = np.linspace(0, x_data[-1], 1000)
x_data_big_datetime = np.array([ts.index[0] + pd.Timedelta(days=int(x)) for x in x_data_big])
y_data = ts.values
o = rb.beast(y_data, start=[y_data[0]], period='1 year', deltat='1/12 year', quiet=True, print_progress=False)
y_data_b = o.season.Y

# stl
from statsmodels.tsa.seasonal import STL, MSTL
res = STL(ts, period=12, seasonal=13).fit()
y_data_s = res.seasonal.values

# Fit the Clausen function to the data using curve_fit
initial_guess = [7, 0.001, -0.5, 365, 1, 0]  # Initial guess for parameters
params, covariance = curve_fit(seasonal_function_fitting, x_data, y_data, p0=initial_guess, maxfev=10000)
params_b, covariance_b = curve_fit(seasonal_function_fitting, x_data, y_data_b, p0=initial_guess, maxfev=10000)
params_s, covariance_s = curve_fit(seasonal_function_fitting, x_data, y_data_s, p0=initial_guess, maxfev=10000)

# Generate curve using fitted parameters
y_fit = seasonal_function_fitting(x_data, *params)
y_fit_b = seasonal_function_fitting(x_data_big, *params_b)
y_fit_s = seasonal_function_fitting(x_data_big, *params_s)

# Plotting
plt.figure(figsize=(20, 20))
# plt.plot(x_data, y_data, label='Data with Noise')
# plt.plot(x_data, y_fit, 'r-', label='Fitted Clausen Function')
plt.plot(ts.index, y_data_b, color='k', linestyle='-', label='Seasonal component BEAST')
plt.plot(ts.index, y_data_s, color='k', linestyle='--', label='Seasonal component STL')
plt.plot(x_data_big_datetime, y_fit_b, color='k', linestyle=':', label=r'Fit BEAST ($R^2>0.99)$')
plt.plot(x_data_big_datetime, y_fit_s, color='k', linestyle='-.', label=r'Fit STL ($R^2=0.95)$')

for year in range(2013, 2023):
    if year == 2013:
        plt.axvspan(datetime(year=year, month=12, day=1), datetime(year=year+1, month=2, day=28), color='#bc5090', alpha=0.2, label='NE Monsoon')
        plt.axvspan(datetime(year=year, month=5, day=1), datetime(year=year, month=9, day=30), color='#58508d', alpha=0.2, label='SW Monsoon')
        plt.axvspan(datetime(year=year, month=3, day=1), datetime(year=year, month=4, day=30), color='#ff6361', alpha=0.2, label='NE-SW Transition')
        plt.axvspan(datetime(year=year, month=10, day=1), datetime(year=year, month=11, day=30), color='#ffa600', alpha=0.2, label='SW-NE Transition')
    else:
        plt.axvspan(datetime(year=year, month=12, day=1), datetime(year=year+1, month=2, day=28), color='#bc5090', alpha=0.2)
        plt.axvspan(datetime(year=year, month=5, day=1), datetime(year=year, month=9, day=30), color='#58508d', alpha=0.2)
        plt.axvspan(datetime(year=year, month=3, day=1), datetime(year=year, month=4, day=30), color='#ff6361', alpha=0.2)
        plt.axvspan(datetime(year=year, month=10, day=1), datetime(year=year, month=11, day=30), color='#ffa600', alpha=0.2)

plt.legend(fontsize=15, loc='upper right')
plt.xlabel('Date', fontsize=20)
plt.ylabel('Amplitude (m)', fontsize=20)
# increase tick size
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

# plt.grid(True)
plt.xlim(datetime(2016, 1, 1), datetime(2018, 12, 31))
# plt.savefig('figures//seasonal_component_fitting.png', dpi=300)
plt.show()

# Print the fitted parameters
print('Fitted Parameters:', params)


Fitted Parameters: [ 1.73173871e+01 -1.41004299e-04 -4.34346102e-01  3.63417120e+02
  1.94248051e-01  1.16525403e+02]


In [117]:
params_b[0], params_s[0]

(14.093981243683753, 15.746914837986012)

In [90]:
# r2 score

from sklearn.metrics import r2_score

r2_score(y_data_b, y_fit_b)


0.9987202066338721

In [282]:
plt.plot(ts.index, y_data - y_fit)
plt.plot(ts.index, y_data - o.season.Y - o.trend.Y)

In [82]:
peaks, _ = find_peaks(y_fit)
minima, _ = find_peaks(-y_fit)


peaks2, _ = find_peaks(y_data)
minima2, _ = find_peaks(-y_data)
# plt.plot(x_data, y_data)

# plt.plot(x_data[peaks], y_data[peaks], 'x')
# plt.plot(x_data[minima], y_data[minima], 'o')



NameError: name 'find_peaks' is not defined

In [265]:
params[2]

-0.4343461021388106

In [269]:
# numeric dates to datetime

import datetime
numeric_dates_fit = np.array([ts.index[0] + datetime.timedelta(days=int(x)) for x in x_data_big])

print(numeric_dates_fit[peaks][0], ts.index[peaks2][0])
print(numeric_dates_fit[minima][0], ts.index[minima2][0])

ts.index[peaks2][3] - ts.index[minima2][2]
numeric_dates_fit[peaks][0] - numeric_dates_fit[minima][0]

# (numeric_dates_fit[peaks][1] - numeric_dates_fit[minima][0]) + abs(numeric_dates_fit[peaks][0] - numeric_dates_fit[minima][0])

2014-03-24 00:00:00+00:00 2014-02-28 00:00:00+00:00
2014-11-12 00:00:00+00:00 2014-04-30 00:00:00+00:00


Timedelta('-233 days +00:00:00')

In [6]:
from IslandTime import plot_shoreline_transects
plot_shoreline_transects(island_info)

In [1]:
import pandas as pd
islands_h = pd.read_excel('Huvadhoo_islands.xlsx', index_col=0)

In [4]:
islands_h[10:]

,island,country,geometry
10,Bodehuttaa,Maldives,POINT (73.572107 0.413156)
11,Boderehaa,Maldives,POINT (73.012414 0.386207)
12,Bolimathaidhoo,Maldives,POINT (73.126432 0.232835)
13,Budhiyahuttaa,Maldives,POINT (73.403026 0.855165)
14,Dhaandhoo,Maldives,POINT (73.46146847985355 0.6209737)
...,...,...,...
182,Veraaviligillaa,Maldives,POINT (73.448828 0.312796)
183,Viligalaa,Maldives,POINT (73.216486 0.51876)
184,Viligili,Maldives,POINT (73.43585 0.756172)
185,Villigalaa,Maldives,POINT (73.129523 0.229659)


In [ ]:
from IslandTime import Workflow, retrieve_island_info

%load_ext autoreload
%autoreload 2

islands_d = []

for isl in islands_h.island[10:]:

    if isl in islands_d:
        continue

    # island_info = retrieve_island_info(island, 'Maldives', verbose=False)

    island_info = Workflow(isl, 'Maldives', execute_analysis=True, execute_segmentation=False, execute_preprocess=False, update_maps=False, overwrite_analysis=True).main()
    islands_d.append(isl)
    # ts = island_info['timeseries_preprocessing']['optimal time period']['dict_timeseries']['coastline_position_transect_5_waterline']['monthly']['coastline_position_transect_67_waterline']

In [ ]:
islands_d

In [ ]:
for i in range(1, 10):
    print(i)
    print(island_info['timeseries_analysis']['coastline_position_transect_{}_waterline'.format(i)]['seasonality']['amplitude_seasonal_BEAST_absrange'])

In [9]:
import matplotlib.pyplot as plt
plt.plot(ts.index, ts.values)

In [243]:
def sin_skewed(x, ks):
    return np.sin(x) / (np.sqrt((ks + np.cos(x))**2 + np.sin(x)**2))

x = np.linspace(0, 2*np.pi, 1000)
ks = -0.8
plt.plot(x, sin_skewed(x, ks))
plt.plot(x, np.sin(x))